In [1]:

from functions_barcelona import posant_accents,utmToLatLng,\
                    ped_to_angles,setmana_a_angles,mes_a_angles,cause_to_angles
import math
import pickle
import pandas as pd
import requests
import json
from datetime import date, datetime, timedelta
from functions_barcelona import getting_daily_weather,getting_next_day

In [ ]:
r = requests.get('https://opendata-ajuntament.barcelona.cat/data/api/3/action/package_search?rows=1000')
response= r.json()


def concatenating_dataframes(filter1):
    
    files=response['result']['results']
    for num, file in enumerate(files):
        
        if ('accidents-' +filter1+'gu') in file['name']:
            print(file['name'])
            filter_list=[]
            
            for fitxer in file['resources']:

                if fitxer['format']=='CSV':

                    print(fitxer['name'],fitxer['format'])
                    #print(fitxer['url'])
                    try:
                        filter_list.append(pd.read_csv(fitxer['url']))
                    except:
                        try:
                            filter_list.append(pd.read_csv(fitxer['url'],encoding='ISO-8859-15'))
                        except:

                            filter_list.append(pd.read_csv(fitxer['url'],sep=';',encoding='ISO-8859-1'))

            column_names=filter_list[0].columns
            data=pd.DataFrame()
            for df in filter_list:
                df=pd.DataFrame.from_records(df.values)
                data=pd.concat([data,df])
                #print(data.shape)
            data.columns=column_names
            data=data.reset_index()
    try:
        return data
    except:
        print('NO FILE WITH THAT FILTER')
    print('CONCATENATING DONE')
#causes_df= concatenating_dataframes('causes-')

noms=['causes-', 'persones-','tipus-', 'vehicles-','']
dict_noms={}
for nom in noms:
    dict_noms[nom[:-1]]=concatenating_dataframes(nom)

pickle.dump( dict_noms, open( "./data/dataframes_dict.pkl", "wb" ) )
#favorite_color = pickle.load( open( "dataframes_dict.pkl", "rb" ) )

## Cleaning ACCIDENTS

In [ ]:
dict_noms = pickle.load( open( "./data/dataframes_dict.pkl", "rb" ) )
acc= dict_noms[''].copy()

#acc.columns
index_to_drop=acc[acc.isnull().sum(axis=1)>10].index
acc=acc.drop(index_to_drop)
acc['Nom_carrer']=acc['Nom_carrer'].fillna('Casetes')
for column in acc:
    if 'Coor' in column:
        print(column)
        acc[column]=[str(coor).replace(',','.') for coor in acc[column]]
        acc[column]= acc[column].astype(float)
for column in acc:
    acc[column]=acc[column].apply(posant_accents)
old_column_names=acc.columns
acc=acc.drop(['Num_postal_caption','Longitud','Latitud','index','Codi_barri', 'Dia_setmana','Mes_any',
             'Descripcio_torn', 'Descripcio_tipus_dia'],axis=1)
print('Number of nulls:',acc.isnull().sum().sum())

acc['Codi_districte']=[x if x!='Desconegut' else -1 for x in acc['Codi_districte']]
acc['Codi_carrer']=acc['Codi_carrer'].astype(int)
#acc['Codi_barri']=[str(x)[-2:].replace('-','') for x in acc.Codi_barri]
tuple_features=[('Nom_carrer','Codi_carrer'),('Nom_districte','Codi_districte')]
for tup in tuple_features:
    if acc[tup[0]].nunique()==acc[tup[1]].nunique():
        acc=acc.drop(tup[1],axis=1)
        print(f"I dropped {tup[1]}")

print(acc.shape)

acc.columns=['num_incident', 'district', 'neighborhood', 'street_code',
       'street_name', 'weekday', 'year', 'month', 'day', 'hour', 'ped_cause',
       'num_deaths', 'num_minorly_injured', 'num_severly_injured',
       'num_victims', 'num_vehicles', 'utm_y',
       'utm_x']
##Translating to English

acc['ped_cause']=acc['ped_cause'].apply(ped_to_angles)
acc['weekday']=acc['weekday'].apply(setmana_a_angles)
acc['month']=acc['month'].apply(mes_a_angles)
acc['num_incident']=[x.strip() for x in acc['num_incident']]
for col in ['year','day','hour','num_deaths','num_minorly_injured','num_severly_injured','num_victims','num_vehicles']:
    acc[col]=acc[col].astype(int)
##eliminatong duplictaes that have 2 ped cause: I am losing the second one.
index_to_drop= acc[acc['num_incident'].duplicated()].index
acc=acc.drop(index_to_drop).reset_index()
acc.to_csv('./data/accidents_only2020.csv')
print('Accidents: ', acc.shape)

##adding causes
causes= dict_noms['causes'].reset_index().copy()


columns_to_keep=['Descripcio_causa_mediata', 'Numero_expedient']

causes=causes[columns_to_keep]
causes.columns=['cause','num_incident']
causes['num_incident']=[x.strip() for x in causes['num_incident']]
causes['cause']=causes.cause.apply(posant_accents).apply(cause_to_angles)
print(causes.shape)
causes=causes.drop_duplicates('num_incident')


causes['cause']=causes['cause'].fillna('Unknown')
causes.to_csv('./data/causes2020.csv')
total= pd.merge(acc,causes, how='left',on='num_incident')
print('causes: ', causes.shape, 'Total: ',total.shape)



##people

people= dict_noms['persones'].reset_index().copy()

columns_to_add=['Numero_Expedient','Desc_Tipus_vehicle_implicat', 'Descripcio_sexe', 'Edat',
                'Descripcio_tipus_persona', 'Descripcio_Lloc_atropellament_vianat',
                'Descripcio_Motiu_desplaçament_vianant',
                'Descripcio_Motiu_desplaçament_conductor', 'Descripcio_victimitzacio',]
people.columns=[posant_accents(col) for col in people.columns]
anual_dict={}
for any_ in sorted(people['NK_ Any'].unique()):
    if any_ in [2010,2011,2012,2013]:
        df=people[people['NK_ Any']==any_].copy()
        df.rename(columns={'Edat':'Descripcio_tipus_persona_','Descripcio_tipus_persona':'Edat_',\
        'Descripcio_Motiu_desplaçament_conductor':'Coordenada_UTM_Y_',
        'Descripcio_Motiu_desplaçament_vianant': 'Coordenada_UTM_X_',
        'Descripcio_Lloc_atropellament_vianat':'Descripcio_victimitzacio_' },inplace=True)
        df=df.drop(df.columns[-5:],axis=1)
        #columnes=[]
        df.columns=[col if col[-1]!='_' else col[:-1] for col in df.columns]
        columnes=[col for col in df.columns if col in columns_to_add]
        df=df[columnes].copy()
        anual_dict[any_]=df
    elif any_ in [2014,2015]:
        df=people[people['NK_ Any']==any_].copy()
        df.rename(columns={'Descripcio_Motiu_desplaçament_vianant':'Coordenada_UTM_Y_','Descripcio_Lloc_atropellament_vianat':'Coordenada_UTM_X_',\
        'Descripcio_tipus_persona':'Descripcio_victimitzacio_',
        'Descripcio_sexe': 'Descripcio_tipus_persona_',
        'Desc_Tipus_vehicle_implicat':'Descripcio_sexe_',
         'Descripcio_causa_vianant':  'Desc_Tipus_vehicle_implicat_'   },inplace=True)
        df=df.drop(df.columns[-6:],axis=1)
        df.columns=[col if col[-1]!='_' else col[:-1] for col in df.columns]
        columnes=[col for col in df.columns if col in columns_to_add]
        df=df[columnes].copy()
        anual_dict[any_]=df
    elif any_ in [2016,2017,2018]:
        df=people[people['NK_ Any']==any_].copy()
        df.rename(columns={'Descripcio_Motiu_desplaçament_vianant':'Descripcio_victimitzacio_','Descripcio_Lloc_atropellament_vianat':'Descripcio_situacio',
        },inplace=True)
        df=df.drop(df.columns[-6:],axis=1)
        df.columns=[col if col[-1]!='_' else col[:-1] for col in df.columns]
        columnes=[col for col in df.columns if col in columns_to_add]
        df=df[columnes].copy()
        anual_dict[any_]=df
    elif any_ in [2019,2020]:
        df=people[people['NK_ Any']==any_].copy()
        columnes=[col for col in df.columns if col in columns_to_add]
        df=df[columnes].copy()
        anual_dict[any_]=df

mapping_columns={'Numero_Expedient':'num_incident',
             'Desc_Tipus_vehicle_implicat':'vehicle',
             'Descripcio_sexe':'gender',
             'Edat':'age',
            'Descripcio_tipus_persona':'people_role',
             'Descripcio_Lloc_atropellament_vianat':'run_over_location',
             'Descripcio_victimitzacio': 'level_injuries',
              'Descripcio_Motiu_desplaçament_vianant':'peds_activity',
            'Descripcio_Motiu_desplaçament_conductor':'drivers_activity'}
people=pd.DataFrame()
for key in anual_dict.keys():
    #print(key, anual_dict[key].shape)
    people= pd.concat([people,anual_dict[key].rename(columns=mapping_columns)])
    #print(people.shape)
    ##too many nulls.
for col in people:
    if people[col].isnull().sum()>10000:
        people=people.drop(col,axis=1)
people=people.drop('level_injuries',axis=1)
people['num_incident']=[x.strip() for x in people['num_incident']]
people['age']=people.age.astype(str)
compressed_people= people.groupby('num_incident').agg(lambda x : ','.join(x)).reset_index()
people.to_csv('./data/people2020.csv')
total= pd.merge(total,compressed_people, how='left',on='num_incident')

total.sample(5)
print('people: ', people.shape, 'Total: ',total.shape)

##TYPE
tipus=dict_noms['tipus'].copy()
type_accident_map={'Atropellament': 'run_over',
         'Col.lisio lateral': 'lateral_collision',
        'Xoc contra element estatic': 'crash_into_stationary',
      'Abast': 'rear-end_collision',
       'Col.lisio frontal':'frontal_collision',
      'Col.lisio fronto-lateral':'frontal-lateral_collision',
      'Caiguda (dues rodes)':'fall--motorcycle',
      'Abast multiple':'multiple_rear-end_collision',
      'Caiguda interior vehicle':'fall_inside_vehicle',
      'Altres':'Other_types',
      'Bolcada (mes de dues rodes)':'overturning',
      'Desconegut':'unknown',
      'Sortida de via amb xoc o col.lisio':'run-off_with_crash_or_collision',
      'Encalç':'rear-end_collision',
      'Sortida de via amb bolcada':'run-off_with_overturning',
      'Xoc amb animal a la calçada':'crash_into_animal_on_road',
      'Resta sortides de via':'run-off_not_included_previously'}

tipus['Descripcio_tipus_accident']=tipus['Descripcio_tipus_accident'].apply(posant_accents).map(type_accident_map)
tipus=tipus[['Numero_expedient','Descripcio_tipus_accident',]].copy()
tipus.columns=['num_incident','accident_type']
tipus['num_incident']=[x.strip() for x in tipus.num_incident]
tipus=tipus.dropna()
tipus=tipus.groupby('num_incident')['accident_type'].agg(lambda x: ','.join(x)).reset_index()
total= pd.merge(total,tipus, how='left',on='num_incident')
tipus.to_csv('./data/types2020.csv')

print('type: ', tipus.shape, 'Total: ',total.shape)

##VEHICLE***Model no surt a gtots els anys

vehicles= dict_noms['vehicles'].copy()
columns_to_add=['Numero_expedient', 'Descripcio_model', 'Descripcio_marca',
       'Descripcio_color', 'Descripcio_carnet', 'Antiguitat_carnet' ]

anual_dict={}

for any_ in sorted(vehicles['NK_Any'].unique()):
    if any_ in [2010,2011,2013,2014,2015,2016,2017]:
        df=vehicles[vehicles['NK_Any']==any_].copy()
        df.rename(columns={'Descripcio_tipus_vehicle':'Descripcio_model_','Descripcio_model':'Descripcio_marca_',\
        'Descripcio_marca':'Descripcio_color_',
        'Descripcio_color': 'Descripcio_carnet_',
        'Descripcio_carnet':'Antiguitat_carnet_' },inplace=True)
        df=df.drop(df.columns[-5:],axis=1)
        df.columns=[col if col[-1]!='_' else col[:-1] for col in df.columns]
        columnes=[col for col in df.columns if col in columns_to_add]
        df=df[columnes].copy()
        anual_dict[any_]=df
    elif any_ in [2018,2019,2020]:
        df=vehicles[vehicles['NK_Any']==any_].copy()        
        columnes=[col for col in df.columns if col in columns_to_add]
        df=df[columnes].copy()
        anual_dict[any_]=df

vehicles=pd.DataFrame()
for key in anual_dict.keys():
    vehicles=pd.concat([vehicles,anual_dict[key]],)
    

vehicles.columns=['num_incident', 'vehicle_model', 'vehicle_brand',
       'vehicle_color', 'type_license', 'signority_license']

vehicles=vehicles.dropna()
vehicles['num_incident']=[x.strip() for x in vehicles.num_incident]
vehicles=vehicles.groupby('num_incident').agg(lambda x: ','.join(x)).reset_index()
vehicles.to_csv('./data/vehicles2020.csv')
total= pd.merge(total,vehicles, how='left',on='num_incident')
total.to_csv('./data/accidents2020.csv')
print('vehicles: ', vehicles.shape, 'Total: ',total.shape)
print('DONE')

## Weather 2020

In [ ]:
##REPEAT THE PROCESS FOR EACH YEAR----I have to limit the number of calls gtherefore I have to do it by year

import datetime
start_date = datetime.date(2020, 1, 1)
end_date = datetime.date(2020, 12, 31)
delta = datetime.timedelta(days=1)
df_total = []
while start_date <= end_date:
    print(start_date)
    df= getting_daily_weather(str(start_date))
    df_total.append(df)
    start_date += delta
pd.concat(df_total).to_csv('./data/weather2020.csv')